In [3]:
import numpy as np 
import time
from time import sleep
from IPython.display import HTML, display, clear_output
import sys

import requests
from bs4 import BeautifulSoup
import json

In [61]:
def get_img(img_file):
    style_str = "width: 600px;"
    html_out = HTML(f"<img style = '{style_str}' src = {img_file}> </img>")
    display(html_out)

In [62]:
files = ["test1.png", "test2.png", "test3.png"]

questions = [
    ["The colour of the triangle is ...?", 
     "What is the shape on the right?",
     "The colour of the shape on the left is ...?"
     ],
    
    ["What shape was purple?", 
     "The shape on the top left is ...?",
     "Which shape was below the circle?", 
     "How many sides does the red shape have?", 
     "What colour was the triangle?"
    ],
    
    ["What was the shape between the star and the pentagon?", 
     "What was the shape above the sun?",
     "The colour of the bottom-right shape is ... ?", 
     "What was the colour of the shape above the smiley face?", 
     "The colour of the circle is ... ?"
     ]
]
          

answers = [
    ["blue", "square", "blue"],
    ["star", "circle", "triangle", "4", "blue"],
    ["square", "heart", "pink", "green", "red"]
]


In [63]:
score = 0

def test(img_file, q_list, a_list):

    global score

    print('You have 12 seconds to look and memorise the table below')
    get_img(img_file)
    time.sleep(12)
    clear_output(wait=False)

    time.sleep(1)

    for i in range(len(q_list)):

        print(f'Your current score is {score} points.')
        
        ans = input(q_list[i])
        if ans.lower() == a_list[i]:
            print("Well done!")
            score += 1+(i*0.5)
            # for each grid, questions get harder due to memory loss as time passes. 0.5 points are added for each consecutive question.
        else: 
            print(f'The correct answer is {a_list[i]}')

       
        time.sleep(1)
        clear_output(wait=False)
        

In [64]:

form_url = "https://docs.google.com/forms/d/e/1FAIpQLSdmrkPwQODW-aQ7o_LlBXIjQKi3sxZqPJ0iSSYOLe1B-VR_Vw/viewform"

def send_to_google_form(data_dict, form_url):

    global name
    global sleep
    global exercise
    
    data_dict = {
        "Name":name,
        "Score":score,
        "Sleep":sleep,
        "Exercise":exercise
    }

    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [65]:
def run_test():

    global score

    score = 0
    
    for i in range(len(files)):
        test(files[i], questions[i], answers[i])
    print(f'Your total score is {score} points.')
    

In [ ]:
consent = input("Please type 'yes' if you consent to your test results to be recorded.")
if consent != "yes":
    print("The test will not continue.")
    time.sleep(1)
    sys.exit()

name = input("Please enter your name.") 

print("How much sleep have you had last night?")
sleep = input("Enter your number of hours as a single number (e.g. 8).") 

print("How many days a week do you exercise for more than 30 minutes?")
exercise = input("Enter your answer as a single number.")

clear_output(wait=False)

run_test()
send_to_google_form(data_dict, form_url)

Please type 'yes' if you consent to your test results to be recorded. yes
